# 主な操作

- Excelファイルは主に、Pandasとopenpyxlの2つで操作する  
- Pandas：データ操作、加工
- openpyxl：Excelの標準規格でファイルを読み書き（計算機能はなく、数式はExcelを起動しないと実行されない）  
参考：[Excelファイルを扱う上での openpyxl と pandas の違い](https://gammasoft.jp/blog/openpyxl-vs-pandas/)

・DEの場合は下記のようなイメージ  
　　openpyxlの関数とFor文を使い、ExcelのSheetをPandas Dataframeとして読み込み  
　→ 読み込んだDataFrameをPandasで加工  
　→ 完成したDataFrameをDBに入れる/Excelファイルとして出力  
　→ Excelファイルの体裁を、openpyxlの機能を使って整える


■Excelファイル読み込み
```python
# Excelファイルを読み込み、Workbookオブジェクトとして変数に格納
wb = openpyxl.load_workbook("ファイルパス")
```

■wbのシート名をリスト形式で出力
```python
wb.sheetnames
# 出力：['0101', '0201', '0301']　※要素はストリング型
```

■wbのワークシートを指定
```python
wb.worksheets[i]
# or
wb.worksheets['ワークシート名']
# 出力：[<Worksheet "0101">]　※要素は「openpyxl.worksheet.worksheet.Worksheet」型
```

■読み込んだすべてのExcelファイルと、すべてのExcelシートをdfに出力して加工
```python
for file in files:
    for sheetname in wb.sheetnames:
        df = pd.read_excel(file, sheet_name=sheetname)
```

■作成したDataFrameをExcelに出力（Excelは新規作成）
```python
df.to_excel('出力するExcelファイルパス', encoding='cp932')
```

In [2]:
import pandas as pd
from pathlib import Path
import shutil
import glob
import os
from openpyxl.styles import Font
from datetime import date
import openpyxl


In [3]:
folder_read = Path(r"\\10.110.0.44\everyone\m03")
files = list(folder_read.glob("*"))
print(files)

file = files[0]
df = pd.read_excel(file, skiprows=1)
df = df[['UDATE', 'PARTS', 'CYC']]
df = df.dropna()
display(df)

[WindowsPath('//10.110.0.44/everyone/m03/220301_a社.xlsx'), WindowsPath('//10.110.0.44/everyone/m03/220301_a社_all.xlsx')]


,UDATE,PARTS,CYC
0,2021-01-01,a001,1
1,2021-01-01,a002,2
2,2021-01-01,a003,2
3,2021-01-01,a004,1
4,2021-01-01,a005,3
5,2021-01-01,a006,3
6,2021-01-01,a007,1


## 複数のシートを読み込みたい場合は、for文で回してPandasでDFに読み取る

In [4]:
wb = openpyxl.load_workbook(file)
wb.sheetnames

['0101', '0201', '0301']

In [5]:
file

WindowsPath('//10.110.0.44/everyone/m03/220301_a社.xlsx')

In [6]:
df_all = pd.DataFrame()

for sn in wb.sheetnames:
    df = pd.read_excel(file, sheet_name=sn, skiprows=1) 
    df = df[['UDATE', 'PARTS', 'CYC']]
    df = df.dropna()
    display(file, df)
    df_all = df_all.append(df).reset_index(drop=True)

df_all

WindowsPath('//10.110.0.44/everyone/m03/220301_a社.xlsx')

,UDATE,PARTS,CYC
0,2021-01-01,a001,1
1,2021-01-01,a002,2
2,2021-01-01,a003,2
3,2021-01-01,a004,1
4,2021-01-01,a005,3
5,2021-01-01,a006,3
6,2021-01-01,a007,1


WindowsPath('//10.110.0.44/everyone/m03/220301_a社.xlsx')

,UDATE,PARTS,CYC
0,2021-02-01,a001,2
1,2021-02-01,a002,3
2,2021-02-01,a003,3
3,2021-02-01,a004,2


WindowsPath('//10.110.0.44/everyone/m03/220301_a社.xlsx')

,UDATE,PARTS,CYC
0,2021-03-01,a001,3.0
1,2021-03-01,a007,2.0


,UDATE,PARTS,CYC
0,2021-01-01,a001,1.0
1,2021-01-01,a002,2.0
2,2021-01-01,a003,2.0
3,2021-01-01,a004,1.0
4,2021-01-01,a005,3.0
5,2021-01-01,a006,3.0
6,2021-01-01,a007,1.0
7,2021-02-01,a001,2.0
8,2021-02-01,a002,3.0
9,2021-02-01,a003,3.0


In [7]:
folder_write = Path(r"./file/Excel")
df_all.to_excel(folder_write/"220301_a社_all.xlsx", encoding='cp932')

In [8]:
"""作成したExcelファイル「220301_a社_all.xlsx」を見やすいように編集して上書き保存""

list_of_files = list(folder_write.glob("*"))
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

wb_edit = openpyxl.load_workbook(latest_file)
ws_e = wb_edit.active

file\Excel\220301_a社_all.xlsx


In [9]:
ws_e.insert_rows(1) 
ws_e.cell(row=1, column=3).value = 'PARTS vs. CYC' 
ws_e.cell(row=1, column=3).font = Font(size=24)
ws_e.cell(row=1, column=4).value = date.today().strftime("%m/%d") 
ws_e.cell(row=1, column=4).font = Font(size=20)
ws_e.freeze_panes = 'A3' 
wb_edit.save(latest_file)

In [10]:
now = datetime.now()
df_all.to_excel(folder_write/'{0:%y%m%d_%H%M}_a社_all.xlsx'.format(now), encoding='cp932')

In [12]:
file_all = latest_file
shutil.copy(file, file_all)
file_all

WindowsPath('file/Excel/220301_a社_all.xlsx')

In [13]:
wb_all = openpyxl.load_workbook(file_all)
wb_all.copy_worksheet(wb_all.worksheets[0])
wb_all.worksheets[-1].title = 'All' 
wb_all.sheetnames

['0101', '0201', '0301', 'All']

In [14]:
ws_all = wb_all['All']

for row in ws_all:
    for cell in row:
        cell.value = None

In [15]:
# シート「All」にDataFrameを貼り付けるために、df_allをリスト型に変換
# 参考：https://biotech-lab.org/articles/4896

values = df_all.to_numpy().tolist()
values.insert(0, df_all.columns.to_list())
values

[['UDATE', 'PARTS', 'CYC'],
 [Timestamp('2021-01-01 00:00:00'), 'a001', 1.0],
 [Timestamp('2021-01-01 00:00:00'), 'a002', 2.0],
 [Timestamp('2021-01-01 00:00:00'), 'a003', 2.0],
 [Timestamp('2021-01-01 00:00:00'), 'a004', 1.0],
 [Timestamp('2021-01-01 00:00:00'), 'a005', 3.0],
 [Timestamp('2021-01-01 00:00:00'), 'a006', 3.0],
 [Timestamp('2021-01-01 00:00:00'), 'a007', 1.0],
 [Timestamp('2021-02-01 00:00:00'), 'a001', 2.0],
 [Timestamp('2021-02-01 00:00:00'), 'a002', 3.0],
 [Timestamp('2021-02-01 00:00:00'), 'a003', 3.0],
 [Timestamp('2021-02-01 00:00:00'), 'a004', 2.0],
 [Timestamp('2021-03-01 00:00:00'), 'a001', 3.0],
 [Timestamp('2021-03-01 00:00:00'), 'a007', 2.0]]

In [16]:
# ws_allにFor文でvaluesの値をセルに書き込んでいく
# 参考：　https://fastclassinfo.com/entry/python_excel_beginners/#7　→「7. シート内のセルの値を一括読み込み・書き出し」
        
for y, row in enumerate(values):
    for x, cell in enumerate(row):
        ws_all.cell(row= y+2, column= x+2, value=values[y][x])

In [17]:
# Excelの上書き保存
wb_all.save(file_all)

## openpyxlを使わず、Pandasの機能だけを使ってすべてのシートを読み込む（dictionaryを使う）【完全に遊び】

In [1]:
df_sheet_all = pd.read_excel(file, sheet_name=None, skiprows=1)
print(type(df_sheet_all))
print(df_sheet_all.keys())

for key in df_sheet_all.keys():
    df = pd.read_excel(file, sheet_name=key, skiprows=1)
    df = df[['UDATE', 'PARTS', 'CYC']]
    df = df.dropna()
    display(df)

NameError: name 'file' is not defined